<a href="https://colab.research.google.com/github/worldone-web/Team4-Traveler/blob/main/BPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 사용자와 음식점을 0,1과 같이 번호로 표현
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/G_Project/philadelphia_checkin.csv")
new_df = df[['newUserId', 'newBusinessId']]

# 번호로만 된 CSV 파일로 저장
new_df.to_csv("/content/drive/MyDrive/G_Project/new_philadelphia_checkin.csv", index=False)

In [ ]:
#10개 이상의 음식점을 방문한 방문자 현황 csv만들기
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("/content/drive/MyDrive/G_Project/new_philadelphia_checkin.csv")

# 각 사용자의 방문 기록 수 계산
user_visit_counts = df['newUserId'].value_counts()

# 방문 기록이 10번 이상인 사용자들의 데이터만 추출
users_10_more_visits = user_visit_counts[user_visit_counts >= 10].index
filtered_df = df[df['newUserId'].isin(users_10_more_visits)]

# 새로운 CSV 파일로 저장
filtered_df.to_csv("/content/drive/MyDrive/G_Project/new_philadelphia_checkin_filtered.csv", index=False)


In [ ]:

#version1
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("/content/drive/MyDrive/G_Project/new_philadelphia_checkin.csv")

# 사용자, 음식점 데이터 추출
user_ids = df['newUserId']
item_ids = df['newBusinessId']

# 각 사용자가 방문한 음식점 목록 생성
visited_restaurants_by_user = {}
for user_id, item_id in zip(user_ids, item_ids):
    if user_id not in visited_restaurants_by_user:
        visited_restaurants_by_user[user_id] = set()
    visited_restaurants_by_user[user_id].add(item_id)

user_id_visits = visited_restaurants_by_user.get(3, set())
for restaurant_id in user_id_visits:
    print(restaurant_id)



1
7
1549
2317
788
6701
1840
1457
5815
1469
3263
6469
220
1757
4961
2279
6512
114
6132
760
5754


In [ ]:
#version2
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("/content/drive/MyDrive/G_Project/new_philadelphia_checkin_filtered.csv")

# 사용자, 음식점 데이터 추출
user_ids = df['newUserId']
item_ids = df['newBusinessId']

# 각 사용자가 방문한 음식점 목록 생성
visited_restaurants_by_user = {}
for user_id, item_id in zip(user_ids, item_ids):
    if user_id not in visited_restaurants_by_user:
        visited_restaurants_by_user[user_id] = set()
    visited_restaurants_by_user[user_id].add(item_id)



In [ ]:
# 각 사용자가 방문한 음식점 목록 출력
for idx, (user_id, visited_restaurants) in enumerate(visited_restaurants_by_user.items()):
    if idx >= 10:
        break
    print(f"User {user_id} visited the following restaurants:")
    for restaurant_id in visited_restaurants:
        print(restaurant_id)
    print()


User 3 visited the following restaurants:
1
7
1549
2317
788
6701
1840
1457
5815
1469
3263
6469
220
1757
4961
2279
6512
114
6132
760
5754

User 8 visited the following restaurants:
3008
1
5159
5100
1613
3342
5357
2480
5677
5973
4314

User 10 visited the following restaurants:
1
3778
6306
1960
3049
4681
5131
2480
3537
6481
3795
436
5012
343
120
763
2302
5087

User 21 visited the following restaurants:
1
1603
4489
3563
4332
3633
466
4690
3988
822
3964

User 22 visited the following restaurants:
1
2182
75
3628
2317
3537
341
3351
4989
1471

User 34 visited the following restaurants:
2304
1
2178
6402
773
3594
779
4236
2189
3087
4496
1683
20
2325
4246
536
6425
4378
31
3615
5793
5029
166
42
2861
1457
437
2230
5820
3006
5950
3777
4296
205
6484
341
85
3796
4951
4952
3934
2912
3581
99
104
4456
6633
6380
4974
5102
6512
6383
3445
5110
5878
2168
2041
4989

User 35 visited the following restaurants:
1
4962
3652
2886
1566
4456
6377
42
5694
2989
3766
3934

User 36 visited the following restaurants:
1
2

In [ ]:
#version 1
# 상호작용 리스트 생성
interactions = []
for user, visited_restaurants in visited_restaurants_by_user.items():
    for restaurant1 in visited_restaurants:
        for restaurant2 in item_ids[~item_ids.isin(visited_restaurants)]:
            interactions.append((user, restaurant1, restaurant2))


In [ ]:
#version2
# 상호작용 리스트 생성
interactions = []
for user, visited_restaurants in visited_restaurants_by_user.items():
    not_visited_restaurants = set(item_ids) - visited_restaurants
    for restaurant1 in visited_restaurants:
        for restaurant2 in not_visited_restaurants:
            interactions.append((user, restaurant1, restaurant2))

KeyboardInterrupt: 

In [ ]:
for interaction in interactions[:100]:
    print(interaction)

In [ ]:
import numpy as np

class MatrixFactorizationBPR:
    def __init__(self, num_users, num_items, num_factors=10, learning_rate=0.01, reg_lambda=0.01): # 모델 초기화 및 하이퍼파라미터 설정

        self.num_users = num_users  # 사용자 수
        self.num_items = num_items  # 아이템 수
        self.num_factors = num_factors  # 잠재 요인 수
        self.learning_rate = learning_rate  # 학습률
        self.reg_lambda = reg_lambda  # 정규화 파라미터
        # 사용자와 아이템의 잠재 요인을 무작위로 초기화
        self.user_factors = np.random.normal(scale=1./num_factors, size=(num_users, num_factors))
        self.item_factors = np.random.normal(scale=1./num_factors, size=(num_items, num_factors))

    def fit(self, interactions, num_epochs=10):
        # 학습 데이터를 사용하여 모델을 학습
        for epoch in range(num_epochs):
            for u, i, j in interactions:
                # 사용자, 아이템, 더미 아이템의 잠재 요인 획득
                user_factor = self.user_factors[u]
                item_factor_i = self.item_factors[i]
                item_factor_j = self.item_factors[j]

                # BPR 손실 함수를 사용하여 그래디언트 계산
                diff = np.dot(user_factor, item_factor_i - item_factor_j)
                gradient = np.exp(-diff) / (1 + np.exp(-diff))

                # 모델 파라미터 업데이트 (SGD)
                self.user_factors[u] += self.learning_rate * (gradient * (item_factor_i - item_factor_j) - self.reg_lambda * user_factor)
                self.item_factors[i] += self.learning_rate * (gradient * user_factor - self.reg_lambda * item_factor_i)
                self.item_factors[j] += self.learning_rate * (-gradient * user_factor - self.reg_lambda * item_factor_j)

    def predict(self, user_id, item_id):
        # 예측 함수
        return np.dot(self.user_factors[user_id], self.item_factors[item_id])

# Example usage
# 모델 인스턴스 생성
num_users = 100
num_items = 50
num_factors = 10
learning_rate = 0.01
reg_lambda = 0.01
model = MatrixFactorizationBPR(num_users, num_items, num_factors, learning_rate, reg_lambda)

# 예제 상호작용 데이터 생성 및 학습
np.random.seed(42)
num_interactions = 1000
interactions = np.random.randint(num_users, size=(num_interactions, 1)).flatten()
interactions = np.column_stack((interactions, np.random.randint(num_items, size=(num_interactions, 2))))
model.fit(interactions)

# 예측
user_id = 0
item_id = 0
prediction = model.predict(user_id, item_id)
print("Prediction for user", user_id, "and item", item_id, ":", prediction)


In [ ]:
import numpy as np

class MatrixFactorizationBPR:
    def __init__(self, num_users, num_items, num_factors=10, learning_rate=0.01, reg_lambda=0.01):
        self.num_users = num_users
        self.num_items = num_items
        self.num_factors = num_factors
        self.learning_rate = learning_rate
        self.reg_lambda = reg_lambda
        self.user_factors = np.random.normal(scale=1./num_factors, size=(num_users, num_factors))
        self.item_factors = np.random.normal(scale=1./num_factors, size=(num_items, num_factors))

    def fit(self, interactions, num_epochs=10):
        for epoch in range(num_epochs):
            for u, i, j in interactions:
                user_factor = self.user_factors[u]
                item_factor_i = self.item_factors[i]
                item_factor_j = self.item_factors[j]

                diff = np.dot(user_factor, item_factor_i - item_factor_j)
                gradient = np.exp(-diff) / (1 + np.exp(-diff))

                self.user_factors[u] += self.learning_rate * (gradient * (item_factor_i - item_factor_j) - self.reg_lambda * user_factor)
                self.item_factors[i] += self.learning_rate * (gradient * user_factor - self.reg_lambda * item_factor_i)
                self.item_factors[j] += self.learning_rate * (-gradient * user_factor - self.reg_lambda * item_factor_j)

    def predict(self, user_id, item_id):
        return np.dot(self.user_factors[user_id], self.item_factors[item_id])

import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("/content/drive/MyDrive/G_Project/new_philadelphia_checkin.csv")

# 사용자, 음식점 데이터 추출
user_ids = df['newUserId']
item_ids = df['newBusinessId']

# 사용자 및 음식점 수
num_users = len(user_ids.unique())
num_items = len(item_ids.unique())

# 각 사용자가 방문한 음식점 목록 생성
visited_restaurants_by_user = {}
for user_id, item_id in zip(user_ids, item_ids):
    if user_id not in visited_restaurants_by_user:
        visited_restaurants_by_user[user_id] = set()
    visited_restaurants_by_user[user_id].add(item_id)

# 상호작용 리스트 생성
interactions = []
for user, visited_restaurants in visited_restaurants_by_user.items():
    for restaurant1 in visited_restaurants:
        for restaurant2 in item_ids[~item_ids.isin(visited_restaurants)]:
            interactions.append((user, restaurant1, restaurant2))

# 모델 인스턴스 생성 및 학습
model = MatrixFactorizationBPR(num_users, num_items, num_factors=10, learning_rate=0.01, reg_lambda=0.01)
model.fit(interactions)

# 예측
user_id = 3  # 예제 사용자
item_id = 1  # 예제 음식점
prediction = model.predict(user_id, item_id)
print("Prediction for user", user_id, "and item", item_id, ":", prediction)
